# Introduction to InterLab contexts internals

**NOTE: This notebooks deals with both standard context use and some low-level aspects of context and their storage.**

InterLab Contexts are a framework for logging, tracing, result storage, and visualization of nested computations and actor interactions.
They have been designed with large textual and structured (e.g. JSON) inputs and outputs in mind, as well as generic and custom visualizations.

A `Context` is an event of certain name, type (`kind`, with small differences in semantics for several built-in ones), start and end time, inputs,
result or error (exception), tags, and any child contexts - forming a rooted tree. Tags allow for context filtering and search. Inputs and result
support structured data and visualizations (see below).

## In InterLab

Contexts are in principle an independent part of InterLab - you can use them in other projects (e.g. for the context browser), or use interlab
without contexts (some contexts are still created but not stored by default, and should pose a trivial performance penalty comparable to logging).

Currently, contexts are stored as JSON files (one JSON file for every designated stored root context), but in the future we plan to also support DB storage.

## Visualization and UI

The framework also contains a web-based browser for the context traces - both directly in a jupyter notebook and in a separate web-browser window.

Contexts and their visualization interact well with dataclasses (and JSON-like data in general) but also support custom visualizations: bitmap images, HTML, SVG (more can be added).

The current browser also shows still running contexts (manual reloads required for refresh).

In [1]:
%load_ext autoreload
%autoreload 2

from dataclasses import dataclass
import json
import matplotlib.pyplot as plt
import interlab
from interlab.context import Context, context, with_context, current_context, Tag
from interlab.lang_models import OpenAiChatModel, AnthropicModel
from interlab.ext.pyplot import capture_figure

def show(obj):
    print(json.dumps(obj, indent=2))

## Basic context examples

### Internals

In [ ]:
# Context with input and output
with Context("MyContext", inputs={"a": 10, "b": 20}) as c:
    c.set_result("Lorem ipsum ...")
    pass

c.display()

In [ ]:
# How it looks as JSON
show(c.to_dict())

In [ ]:
# Context nesting
with Context("MyContext") as c:
    with Context("ChildContext1") as c1:
        c1.set_result("result")
    with Context("ChildContext2") as c2:
        c2.set_result("result2")
    c.set_result("")
c.display()

### Functions

In [ ]:
# Context decorator

@with_context
def my_function(a, b):
    return a + b

with Context("root") as c:
    my_function(10, 20)
    
c.display()

### Exceptions

In [ ]:
# Error handling

@with_context
def my_function(a, b):
    raise Exception("Oops")

try:
    with Context("root") as c:
        my_function(10, 20)
except:
    pass

c.display()

In [ ]:
# note: errors must still be handled!

# This is expected to throw an error - note the context would still be stored if you set up storage here
with Context("root") as c:
    my_function(1, 2)

In [ ]:
c.display()

### Dataclasses

In [ ]:
# Dataclasses are serialized for logging inside contexts

@dataclass
class Person:
    name: str
    age: int
    
@with_context
def say_hi(person):
    return f"Hi {person.name}!"

with Context("root") as c:
    person = Person("Alice", 21)
    say_hi(person)

c.display()

In [ ]:
# same with dataclass outputs
@with_context
def give_birth(name):
    return Person(age=0, name=name)

with Context("root") as c:
    give_birth("Alice")

c.display()

### LLMs

Note that you need to have the LLM API keys [stored in an `.env` file](https://github.com/theskumar/python-dotenv#getting-started) (recommended), or as environment variables. (Storing API keys in this notebook is possible but unadvisable for security reasons.)

In [2]:
# Load API keys from ".env" file if you have one
import dotenv
dotenv.load_dotenv()

# LLM queries track context automatically
model = OpenAiChatModel()

with Context("root") as c:
        response = model.query("How are you?")
        model.query("Is the following text generated by an LLM?\n\n" + response)
    
c.display()

In [ ]:
# async LLM queries work as well
import asyncio 

model1 = OpenAiChatModel()
model2 = AnthropicModel()

@with_context
async def make_queries(model):
    response = await model.aquery("Hi are you?")
    return await model.aquery("Is this nice response?\n\n" + response)  

with Context("root") as c:
    q1 = make_queries(model1)
    q2 = make_queries(model2)

    await q1
    await q2

c.display()

### Tags

In [ ]:
# Tags

with Context("root", tags=["tag1", "tag2"]) as c:
    c.add_tag("exp1")  # Add to a context dynamically
    c.add_tag(Tag("success!", color="lightgreen"))  # Add colored tag

c.display()

### Images and Plots

In [ ]:
with Context("root") as root:
    with Context("first", meta={"color": "lightgreen"}):
        pass
    with Context("second", meta={"color": "lightblue"}):
        pass        

    fig, ax = plt.subplots()
    
    fruits = ['apple', 'blueberry', 'cherry', 'orange']
    counts = [40, 100, 30, 55]
    bar_labels = ['red', 'blue', '_red', 'orange']
    bar_colors = ['tab:red', 'tab:blue', 'tab:red', 'tab:orange']
    
    ax.bar(fruits, counts, label=bar_labels, color=bar_colors)
    
    with Context("image demo", inputs={"my_chart": capture_figure(), "other_param": 42}) as root2:
        pass

root.display()

In [ ]:
# get current context

with Context("root") as c:
    with Context("child"):
        current_context().add_tag("tag1")

c.display()

### Events

In [ ]:
# Events (instant context with immediate result)

with Context("root") as c:
    c.add_event("Message to Alice", kind="message", data="Hi, Alice!")

c.display()

### Storage

In [ ]:
# Register context into storage
storage=interlab.context.FileStorage("/tmp/interlab")
with Context("root1", storage=storage):
    pass


In [ ]:
# Manual writing context into storage

with Context("root2", tags=["hello"]) as c:
    pass

storage.write_context(c)

In [ ]:
# with block and storage; all root context in storage block are written in storage

with storage:
    with Context("root3"):  # <-- This context is root; it is automatcally written in storage
        with Context("child"): # <-- This context is not root; not automatically written in storage
            pass

In [ ]:
# Composing directory structure with contexts:
# each context subtree marked with `directory=True` getns stored in a separete json rather than in the main json

with Context("root4", storage=storage, directory=True) as root:
    with Context("first child", directory=True):
        with Context("a"):
            pass
        with Context("b", tags=["hello"]):
            pass
    with Context("second child", directory=True):
        with Context("a"):
            pass
        with Context("b"):
            pass

import pathlib
list(pathlib.Path(f"{storage.directory}/{root.uid}.ctx").rglob("*"))

### Data browser

In [ ]:
# Running data browser over storage

storage.start_server()

In [ ]:
# Show live content of storage
storage.live_display()

In [ ]:
# Long running cell, you can observe it in browser in running state (nneds to be refreshed manually)

with Context("Long running", storage=storage):
    with Context("Child1"):
        import time
        time.sleep(10)

In [ ]:
# Read all stored (root) contexts

for context in storage.read_all_contexts():
    print(context.uid, context.name)

In [ ]:
# Recursively search for specific contexts

for context in storage.find_contexts(lambda ctx: ctx.has_tag_name("hello")):
    print(context.uid, context.name)

In [ ]:
# Read a context by uid

context = storage.read_context(root.uid)
print(context.uid, context.name)


# Search in a given context

context.find_contexts(lambda x: x.name == "a")